In [1]:
import pandas as pd
from orderbook_simulator.visualize.processus import simulate_brownian_geo, plot_price_series, simulate_brownian_pur

# Brownien pur

In [6]:
serie_bp = simulate_brownian_pur(n=20000, x0=100.0, mu=0.02, sigma=0.6, seed=42)
plot_price_series(serie_bp, title="Brownien pur (additif)")

# Brownien géométrique

In [8]:
serie = simulate_brownian_geo(n=2000, p0=100.0, drift=0.0002, vol=0.01, seed=42)
plot_price_series(serie, title="Brownien géométrique")

In [41]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from dataclasses import dataclass
from typing import List, Optional, Tuple

@dataclass
class Phase:
    name: str
    n: int           # nombre de pas
    mu: float        # drift par pas (log-retour moyen)
    sigma: float     # vol par pas (écart-type des log-retours)

def simulate_gbm_segment(p0: float, n: int, mu: float, sigma: float, rng: np.random.Generator) -> np.ndarray:
    """GBM discret: P_{t+1} = P_t * exp(mu + sigma * eps_t). Retourne un array de longueur n."""
    r = rng.normal(loc=mu, scale=sigma, size=n)
    return p0 * np.exp(np.cumsum(r))

def simulate_breakout_cycle(
    phases: List[Phase],
    p0: float = 100.0,
    seed: Optional[int] = 0
) -> Tuple[pd.Series, pd.DataFrame]:
    """Concatène des segments GBM selon les phases. Retourne la série prix et un tableau des bornes de phases."""
    rng = np.random.default_rng(seed)
    prices = []
    bounds = []
    start_idx = 1
    last_price = p0
    for ph in phases:
        seg = simulate_gbm_segment(last_price, ph.n, ph.mu, ph.sigma, rng)
        prices.append(seg)
        end_idx = start_idx + ph.n - 1
        bounds.append(dict(name=ph.name, start=start_idx, end=end_idx))
        last_price = seg[-1]
        start_idx = end_idx + 1
    series = pd.Series(np.concatenate(prices), index=pd.RangeIndex(1, sum(p.n for p in phases)+1), name="price")
    bounds_df = pd.DataFrame(bounds)
    return series, bounds_df

def plot_cycle(series: pd.Series, bounds_df: pd.DataFrame, title: str = "Cycle de breakout (GBM par phases)"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=series.index, y=series.values, mode="lines", name="prix"))
    # Zones de phase
    for i, row in bounds_df.iterrows():
        fig.add_vrect(
            x0=row["start"], x1=row["end"],
            fillcolor=None, line_width=0, opacity=0.08,
            annotation_text=row["name"], annotation_position="top left"
        )
        # Trait fin aux frontières
        fig.add_vline(x=row["start"], line_width=1, opacity=0.2)
    fig.update_layout(
        title=title,
        xaxis_title="Pas de temps",
        yaxis_title="Prix",
        margin=dict(l=40, r=20, t=40, b=40),
        showlegend=False,
    )
    fig.show()

# %% Paramètres par défaut d'un cycle
phases = [
    Phase("1) Compression", n=150, mu=0.0000, sigma=0.003),   # plat, faible vol
    Phase("2) Breakout",    n=60,  mu=0.0040, sigma=0.015),   # impulsion forte
    Phase("3) Extension",   n=120, mu=0.0010, sigma=0.008),   # poursuite modérée
    Phase("4) Retour",      n=120, mu=-0.0007, sigma=0.005),  # retour doux / consolidation
]

# %% Simulation + affichage
serie, bornes = simulate_breakout_cycle(phases*2, p0=5.0, seed=42)
plot_cycle(serie, bornes, title="Cycle de breakout: Compression → Breakout → Extension → Retour")